In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

C:\Users\almad\anaconda3\envs\tensorflow2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [3]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    version=1.0.0,
    description='The MNIST database of handwritten digits.',
    urls=['https://storage.googleapis.com/cvdf-datasets/mnist/'],
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

In [4]:
train, test = dataset['train'], dataset['test']

num_validation_samples = int(0.1 * mnist_info.splits['train'].num_examples)
num_validation_samples

6000

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

train_validation_scaled = train.map(scale)
test_scaled = test.map(scale)

BUFFER_SIZE = 10000
BATCH_SIZE = 50

train_validation_shuffled = train_validation_scaled.shuffle(BUFFER_SIZE)

validation_data = train_validation_shuffled.take(num_validation_samples)
train_data = train_validation_shuffled.skip(num_validation_samples)

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_scaled.batch(mnist_info.splits['test'].num_examples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model

In [6]:
input_size = 784
output_size = 10
hidden_layer_size = 250

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

## Optimizer and the loss function

In [7]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training

In [8]:
NUM_EPOCHS = 10

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/10
1080/1080 - 11s - loss: 0.2171 - accuracy: 0.9354 - val_loss: 0.1440 - val_accuracy: 0.9587 - 11s/epoch - 10ms/step
Epoch 2/10
1080/1080 - 8s - loss: 0.0864 - accuracy: 0.9736 - val_loss: 0.0784 - val_accuracy: 0.9768 - 8s/epoch - 7ms/step
Epoch 3/10
1080/1080 - 8s - loss: 0.0608 - accuracy: 0.9814 - val_loss: 0.0653 - val_accuracy: 0.9800 - 8s/epoch - 7ms/step
Epoch 4/10
1080/1080 - 8s - loss: 0.0473 - accuracy: 0.9853 - val_loss: 0.0505 - val_accuracy: 0.9852 - 8s/epoch - 7ms/step
Epoch 5/10
1080/1080 - 7s - loss: 0.0371 - accuracy: 0.9876 - val_loss: 0.0445 - val_accuracy: 0.9862 - 7s/epoch - 7ms/step
Epoch 6/10
1080/1080 - 7s - loss: 0.0294 - accuracy: 0.9909 - val_loss: 0.0359 - val_accuracy: 0.9885 - 7s/epoch - 7ms/step
Epoch 7/10
1080/1080 - 7s - loss: 0.0288 - accuracy: 0.9903 - val_loss: 0.0275 - val_accuracy: 0.9923 - 7s/epoch - 7ms/step
Epoch 8/10
1080/1080 - 9s - loss: 0.0208 - accuracy: 0.9929 - val_loss: 0.0406 - val_accuracy: 0.9895 - 9s/epoch - 8ms/step
Epoch

## Testing

In [9]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 738ms/step - loss: 0.0828 - accuracy: 0.9789


In [10]:
print(f'Test loss: {round(test_loss, 2)}. Test Accuracy: {round(test_accuracy*100, 1)}%')

Test loss: 0.08. Test Accuracy: 97.9%


## Converting to TFLite 

In [11]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('model.tflite', 'wb') as f:  
	f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\almad\AppData\Local\Temp\tmp5uj2aycs\assets


INFO:tensorflow:Assets written to: C:\Users\almad\AppData\Local\Temp\tmp5uj2aycs\assets


#### Predictions with TFLite

In [12]:
# First thing you need to do is to create an instance of TFLite interpreter
# and allocate the input and output tensors to it.
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

In [13]:
# Since TFLite is a bit low-level, we will need to provide the interpreter (model) 
# with an input/output index when making predictions.

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

